# Imports & Environment

In [30]:
import os
import pandas as pd
import numpy as np
import torch
import sentence_transformers
import openai

In [31]:
def identify_tensor_device():
    if torch.backends.mps.is_available():
        device = "mps"
    elif torch.cuda.is_available():
        device = "cuda"
    else:
        device = "cpu"
    return device


In [32]:
device = identify_tensor_device()
device

'mps'

In [128]:
#model = sentence_transformers.SentenceTransformer('msmarco-distilbert-base-v4', device=device)
#model = sentence_transformers.SentenceTransformer('msmarco-distilbert-base-v4', device=device)

# Searching the corpus


In [7]:
mr_archive = pd.read_csv('output/mr_archive.csv')

In [64]:
mr_archive.head()

,title,author,publish_date,content,outlinks,tags,link,local_path
0,Self-punishment and incentives,Tyler Cowen,2013-03-30 07:03:53,\n[LeBron] James told me that when he was work...,['http://www.grantland.com/story/_/id/9109245/...,[],https://marginalrevolution.com/marginalrevolut...,/Users/jvm/Development/web_crawling/marginal_r...
1,Assorted links,Tyler Cowen,2013-03-30 13:21:30,\n1. On nitrogen fertilizer.\n2. Roberta Smith...,['http://www.inexactchange.org/blog/2013/03/28...,[],https://marginalrevolution.com/marginalrevolut...,/Users/jvm/Development/web_crawling/marginal_r...
2,A Brilliant New Method of Price Discrimination...,Alex Tabarrok,2013-03-30 12:32:59,"\nTo maximize profit, airlines want to charge ...",['http://www.worthpublishers.com/catalog/stati...,[],https://marginalrevolution.com/marginalrevolut...,/Users/jvm/Development/web_crawling/marginal_r...
3,*Simpler: The Future of Government*,Tyler Cowen,2013-03-31 18:52:19,\nThat is from Cass Sunstein (always worth rea...,['https://www.amazon.com/Simpler-Government-Ca...,[],https://marginalrevolution.com/marginalrevolut...,/Users/jvm/Development/web_crawling/marginal_r...
4,Assorted links,Tyler Cowen,2013-03-31 07:48:35,"\n1. Velocity maps.\n2. Carmina Burana, sung b...",['http://www.fastcoexist.com/1681677/a-new-map...,[],https://marginalrevolution.com/marginalrevolut...,/Users/jvm/Development/web_crawling/marginal_r...


In [209]:
#model = sentence_transformers.SentenceTransformer('multi-qa-mpnet-base-dot-v1', device=device)
model = sentence_transformers.SentenceTransformer('all-MiniLM-L12-v2', device=device)


In [178]:
embedding_test_unbatched = model.encode('alpha', show_progress_bar=True)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

NotImplementedError: The operator 'aten::cumsum.out' is not current implemented for the MPS device. If you want this op to be added in priority during the prototype phase of this feature, please comment on https://github.com/pytorch/pytorch/issues/77764. As a temporary fix, you can set the environment variable `PYTORCH_ENABLE_MPS_FALLBACK=1` to use the CPU as a fallback for this op. WARNING: this will be slower than running natively on MPS.

In [ ]:
embedding_test_batched = model.encode(['alpha', 'square', 'anonymity'], show_progress_bar=True)

In [176]:
print('Self-match:', sentence_transformers.util.dot_score(embedding_test_unbatched, embedding_test_batched[0]))
print('Other-match:', sentence_transformers.util.dot_score(embedding_test_unbatched, embedding_test_batched[1]))
print('Other-match:', sentence_transformers.util.dot_score(embedding_test_unbatched, embedding_test_batched[2]))

Self-match: tensor([[0.9914]])
Other-match: tensor([[0.1675]])
Other-match: tensor([[0.1760]])


In [105]:
corpus_embeddings = torch.tensor(torch.load('output/mr_embeddings.pt')).to(device)

In [106]:
print(concatenate_title_text(mr_archive.iloc[test_index]))

Title: Self-punishment and incentives
    Tags: [, ]
    Content: 
[LeBron] James told me that when he was working on his 3s, he’d punish himself until he met a lofty set of self-enforced shooting milestones.
“It’s work,” James says. “It’s a lot of work. It’s being in workouts, and not accomplishing your goal, and paying for it. So, if I get to a spot in a workout and want to make eight out of 10, if I don’t make eight of 10, then I run. I push myself to the point of exhaustion until I make that goal. So you build up that mentality that you got to make that shot and then use that in a game situation — it’s the ultimate feeling, when you’re able to work on something and implement it.”
Here is more, all of it focused on how LeBron James improved his game.



In [100]:
model.encode(concatenate_title_text(mr_archive.iloc[test_index]))

array([ 4.62842137e-01,  4.17626470e-01, -2.56969243e-01,  6.38134703e-02,
        7.91084617e-02,  3.65754992e-01,  5.70676625e-01, -5.32728553e-01,
       -3.22708815e-01,  8.84878188e-02,  2.23417342e-01,  6.02718532e-01,
        5.37983030e-02, -1.25656009e-01,  1.65926144e-01, -2.63852686e-01,
        5.65725803e-01, -1.90581173e-01,  4.99700397e-01, -6.15747154e-01,
       -2.02796027e-01,  1.13386166e+00,  2.06236050e-01,  1.88513979e-01,
       -3.05607796e-01, -3.37160438e-01,  2.45144531e-01, -1.97676182e-01,
       -6.48346364e-01,  2.77979344e-01,  2.04377010e-01, -3.49896550e-01,
        2.18025595e-01, -3.41442049e-01, -3.91747534e-01, -5.32011151e-01,
        1.00993842e-01, -9.40842852e-02,  8.64315778e-02, -4.75268453e-01,
        2.14811727e-01,  1.93131611e-01,  7.42075890e-02,  1.89245433e-01,
        1.16290501e-03,  3.29866379e-01, -1.67530492e-01, -8.22877884e-02,
        3.44126858e-02, -1.43611237e-01, -4.16159958e-01,  1.64211646e-01,
       -5.71306884e-01, -

In [107]:
# Sanity check test
# This should be close to zero

def concatenate_title_text(row):
    return f'''Title: {row["title"]}
    Tags: {', '.join(row["tags"])}
    Content: {row["content"]}'''

test_index = 0
np.sum(np.abs(model.encode(concatenate_title_text(mr_archive.iloc[test_index])) - corpus_embeddings[test_index].cpu().numpy()))


4.328331

In [10]:
corpus_embeddings.shape

(3974, 768)

In [11]:
query_embedding = model.encode('a', convert_to_tensor=True)

In [25]:
cos_scores = sentence_transformers.util.cos_sim(query_embedding, corpus_embeddings)[0]

In [108]:
def search_mr_for_query(query):
    query_embedding = model.encode(query, convert_to_tensor=True)

    cos_scores = sentence_transformers.util.cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=5)
    
    return top_results

### Reasonably good queries
* Why did Brexit happen? (2nd hit is Why Brexit happened and what it means)
* Should we believe in the recent reports about UFOs?
* Does the minimum wage cause disemployment?

### Bad-ish

* What are the best Austrian novels?

### Bad
* Why did Europe diverge from the rest of the world?

In [38]:
query = 'Does the minimum wage cause disemployment?'
results = search_mr_for_query(query)

In [39]:
results

torch.return_types.topk(
values=tensor([0.5580, 0.5496, 0.5241, 0.5236, 0.5227], device='mps:0'),
indices=tensor([3038,  572, 3275, 1983, 2657], device='mps:0'))

In [40]:
results.indices[0].item()

3038

In [41]:
for i in range(len(results[0])):
    print(f'Result #{i+1}')
    print(f'Score: {results.values[i]}')
    hit = mr_archive.iloc[results.indices[i].item()]
    print(f'Title: {hit["title"]}')
    print(f'Author: {hit["author"]}')
    print(f'Date: {hit["publish_date"]}')
    print(f'Content: {hit["content"]}')
    print(f'Link: {hit["link"]}')
    print()
    #result.indices[0]
    #mr_archive.iloc[1939]

Result #1
Score: 0.557997465133667
Title: The Minimum Wage in an Online Job Market
Author: Alex Tabarrok
Date: 2018-07-31 07:27:33
Content: 
John Horton has written a novel paper that uses an experiment and a policy change in an online job market to understand the effects of the minimum wage. The job market in question is something like the Upwork platform where firms can post jobs and workers from anywhere in the world can post offers to work at an hourly wage to do tasks such as computer programming, data entry, design and transcription. Typically workers are hired for a week or two.
Horton was able to implement a minimum wage by simply not allowing a worker to offer to work at less than the minimum wage for a randomly chosen set of jobs.
During the experimental period, firms posting an hourly job opening were immediately assigned to an experimental cell. The experiment consisted of four experimental cells: a control group with the platform status quo of no minimum wage, which receiv

In [256]:
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

In [257]:
def get_gpt2_token_count(text):
    return np.sum(tokenizer(text)['attention_mask'])

In [258]:
def encode_hit_for_gpt3(hit):
    return f"""Title: {hit["title"]}
    Tags: {hit["tags"]}
    Content: {hit["content"]}
    """
def generate_query_for_gpt3(hits, query):
    hits_encoded = [encode_hit_for_gpt3(hit) for hit in hits]
    token_counts = [get_gpt2_token_count(hit) for hit in hits_encoded]
    if np.sum(token_counts) > 3000:
        assert False, f"Probably too many tokens: {np.sum(token_counts)}"
    article_prompt = f"""A user queried: '{query}'
    
    Please read the following articles and attempt to answer the query.
    
    """
    for i, hit in enumerate(hits_encoded):
        article_prompt += f"""
        Article number {i+1}:
        
        {hit}
        """
    
    article_prompt += f"""Based on the above, in detail, how would you respond to the user's query, '{query}'"""
    return article_prompt
    

In [42]:
hits = [mr_archive.iloc[ix.item()] for ix in results.indices]
gpt3_prompt = generate_query_for_gpt3(hits, query)

In [43]:
get_gpt2_token_count(gpt3_prompt)

2080

In [44]:
gpt3_prompt

"A user queried: 'Does the minimum wage cause disemployment?'\n    \n    Please read the following articles and attempt to answer the query.\n    \n    \n        Article number 1:\n        \n        Title: The Minimum Wage in an Online Job Market\n    Tags: []\n    Content: \nJohn Horton has written a novel paper that uses an experiment and a policy change in an online job market to understand the effects of the minimum wage. The job market in question is something like the Upwork platform where firms can post jobs and workers from anywhere in the world can post offers to work at an hourly wage to do tasks such as computer programming, data entry, design and transcription. Typically workers are hired for a week or two.\nHorton was able to implement a minimum wage by simply not allowing a worker to offer to work at less than the minimum wage for a randomly chosen set of jobs.\nDuring the experimental period, firms posting an hourly job opening were immediately assigned to an experimenta

In [45]:
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")


In [46]:
response = openai.Completion.create(model="text-davinci-002", prompt=gpt3_prompt, temperature=.5, max_tokens=200)


In [47]:
response

<OpenAIObject text_completion id=cmpl-5tjcfacOndoNNpqQGejTWsMPefnMP at 0x2a549ecc0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\n\nThe answer to this query is not entirely clear. There is evidence that suggests that the minimum wage can cause disemployment, especially among low-wage workers and those who are less skilled or educated. However, there is also evidence that suggests that the minimum wage may not have a large effect on employment overall, and that it may actually lead to more productive workers being hired. Ultimately, more research is needed in order to determine the full extent of the minimum wage's impact on employment."
    }
  ],
  "created": 1663965097,
  "id": "cmpl-5tjcfacOndoNNpqQGejTWsMPefnMP",
  "model": "text-davinci-002",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 98,
    "prompt_tokens": 1905,
    "total_tokens": 2003
  }
}

In [ ]:
.002